In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.targeted_attack import Nettack

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
data_dr = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading cora dataset...
Selecting 1 largest connected components


In [6]:
degrees = np.array(adj.sum(axis=1)).flatten()
max_degree_vertex = degrees.argmax()

In [7]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [8]:
ptb_rate = 0.1

In [9]:
model = Nettack(model=surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(features, adj, labels, max_degree_vertex, n_perturbations=perturbations)
modified_adj = model.modified_adj

##### Starting attack #####
##### Attack only using structure perturbations #####
##### Attacking the node directly #####
##### Performing 506 perturbations #####
##### ...1/506 perturbations ... #####


/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/numba/core/ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../anaconda3/envs/mesp/lib/python3.8/site-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


##### ...2/506 perturbations ... #####
##### ...3/506 perturbations ... #####
##### ...4/506 perturbations ... #####
##### ...5/506 perturbations ... #####
##### ...6/506 perturbations ... #####
##### ...7/506 perturbations ... #####
##### ...8/506 perturbations ... #####
##### ...9/506 perturbations ... #####
##### ...10/506 perturbations ... #####
##### ...11/506 perturbations ... #####
##### ...12/506 perturbations ... #####
##### ...13/506 perturbations ... #####
##### ...14/506 perturbations ... #####
##### ...15/506 perturbations ... #####
##### ...16/506 perturbations ... #####
##### ...17/506 perturbations ... #####
##### ...18/506 perturbations ... #####
##### ...19/506 perturbations ... #####
##### ...20/506 perturbations ... #####
##### ...21/506 perturbations ... #####
##### ...22/506 perturbations ... #####
##### ...23/506 perturbations ... #####
##### ...24/506 perturbations ... #####
##### ...25/506 perturbations ... #####
##### ...26/506 perturbations ... #####
##### ..

### arch tests

In [19]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [20]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges.pth"

In [21]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [24]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [25]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [26]:
arr_diff

<2485x2485 sparse matrix of type '<class 'numpy.float32'>'
	with 740 stored elements in COOrdinate format>

In [27]:
data_modded = copy.deepcopy(data)

In [28]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

10556
740


In [29]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")

Evasion Change in Accuracy: 0.1241%
Evasion Change in Loss: -1.9890%
Epoch 0, Train Loss - 1.948091745376587, Val Loss - 1.9087685346603394, Val Accuracy - 0.278
Epoch 20, Train Loss - 0.3090420067310333, Val Loss - 0.854448139667511, Val Accuracy - 0.752
Epoch 40, Train Loss - 0.08178330212831497, Val Loss - 0.8038424253463745, Val Accuracy - 0.748
Epoch 60, Train Loss - 0.05486995354294777, Val Loss - 0.7635618448257446, Val Accuracy - 0.75
Epoch 80, Train Loss - 0.055871471762657166, Val Loss - 0.7592857480049133, Val Accuracy - 0.758
Epoch 100, Train Loss - 0.041589248925447464, Val Loss - 0.7715467810630798, Val Accuracy - 0.756
Epoch 120, Train Loss - 0.04868235066533089, Val Loss - 0.7865746021270752, Val Accuracy - 0.76
Epoch 140, Train Loss - 0.04342997446656227, Val Loss - 0.8184025883674622, Val Accuracy - 0.756
Epoch 160, Train Loss - 0.0416611023247242, Val Loss - 0.8139201998710632, Val Accuracy - 0.752
Epoch 180, Train Loss - 0.020193591713905334, Val Loss - 0.8173336982